In [ ]:

import os 
import requests 
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown,display
from openai import OpenAI

In [ ]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI()

In [ ]:
headers = {  
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"  
}  # Custom headers to mimic a real browser request  

class Website_scraper:  
    def __init__(self, url):  
        """Create a Website object from the given URL using BeautifulSoup"""  
        self.url = url  
        response = requests.get(url, headers=headers)  # Send GET request to the URL  
        soup = BeautifulSoup(response.content, 'html.parser')  # Parse HTML content  
        self.title = soup.title.string if soup.title else "No title found"  # Extract title  
    
        for irrelevant in soup.body(["script", "style", "img", "input"]):  
            irrelevant.decompose()  # Remove unwanted elements  
        
        self.text = soup.body.get_text(separator="\n", strip=True)  # Extract clean text  

In [ ]:
system_prompt = "You are an assistant that analyzes and find contacts from website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."                                                                                                                                                                  

In [ ]:
# function 01: 

def user_prompt_for(website_scraper):
    user_prompt = f"you are looking at a website titled {website_scraper.title}"
    user_prompt += "\n the contents of the website is as follows: \
please provide important contact of this website in markdown.\
if it includes methos for contacting, then summarize these too.\n\n"
    user_prompt += website_scraper.text
    return user_prompt 

In [ ]:
# function 02: 
def prompt_payload(website_scraper):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website_scraper)}
    ]


In [ ]:
# function 03:
def gpt_response(url):
    website_scraper = Website_scraper(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = prompt_payload(website_scraper)
    )
    return response.choices[0].message.content


In [ ]:
# function 04: 
def display_gpt_response(url):
    response = gpt_response(url)
    display(Markdown(response))


In [ ]:
# call the result 
display_gpt_response("https://and-element.com/")
